# A mobile app for lottery addiction

A medical institute that aims to prevent and treat gambling addictions wants to build a dedicated mobile app to help lottery addicts better estimate their chances of winning. The institute has a team of engineers that will build the app, but they need us to create the logical core of the app and calculate probabilities.

For the first version of the app, they want us to focus on the 6/49 lottery and build functions that enable users to answer questions like:

- What is the probability of winning the big prize with a single ticket?
- What is the probability of winning the big prize if we play 40 different tickets (or any other number)?
- What is the probability of having at least five (or four, or three, or two) winning numbers on a single ticket?

### Core Functions

In the 6/49 lottery, six numbers are drawn from a set of 49 numbers that range from 1 to 49. The drawing is done without replacement, which means once a number is drawn, it's not put back in the set.

We'll use two principal function for cal

In [1]:
def factorial(n):
    """
    Calculate the factorial of n (n!)
    Args:
        n: numeric value
    Returns:
        The n!.
    Examples
    ________
    >>> factorial(5)
    120
    """
    if n == 2:
        return 2
    return n * factorial(n - 1)

In [2]:
def combinations(n, k):
    """
    Calculate the combinations of n taking k objects.
    Args:
        n: number represent a group.
        k: number of takin objectcs from group.
    Returns:
        The nCk.
    Examples
    ________
    >>> combinations(52, 5)
    2598960.0
    """
    return factorial(n) / (factorial(k) * factorial(n - k))
